In [80]:
import os
from itertools import product
from dataclasses import dataclass, field
from sentence_transformers import SentenceTransformer, util
from typing import List

In [39]:
repo_path = os.path.dirname(os.getcwd())
summaries_path = os.path.join(repo_path, 'cvs', 'summary.txt')
descriptions_path = os.path.join(repo_path, 'cvs', 'job_descriptions.txt')

In [40]:
with open(summaries_path, 'r') as file:
    txt = file.read()
    summaries = txt.split('\n\n-------------------\n\n')
summaries.pop(-1)

with open(descriptions_path, 'r') as file:
    txt = file.read()
    job_descriptions = txt.split('\n\n-------------------\n\n')

In [107]:
@dataclass
class Candidate_Selector():
    candidates: List[str] = field(default=None)
    job_description: str = field(default=None)
    model: str = 'paraphrase-multilingual-MiniLM-L12-v2'

    def make_model(self):
        return SentenceTransformer(self.model)
    
    def embed(self, data):
        embedding = self.make_model().encode(data)
        return embedding
    
    def calculate_similarities(self):
        candidates = self.embed(self.candidates)
        job = self.embed(self.job_description)
        hits = util.semantic_search(job, candidates)

        return hits
    
    
    def select_top_candidates(self, k = 1):
        hits = self.calculate_similarities()
        if k == 1:
            return hits[0][:k]['corpus_id']
        top_k = []
        for i in range(k):
            top_k.append(hits[0][i]['corpus_id'])
        return top_k

    def print_top_candidates(self, k = 1):
        top_k = self.select_top_candidates(k=k)
        result = f"""Job desciption: {self.job_description}

        Top {k} candidate summaries:

        {'\n\n------\n\n'.join([self.candidates[k] for k in top_k])}
        """
        print(result)

In [108]:
selector = Candidate_Selector(summaries, job_descriptions[0])

In [109]:
selector.print_top_candidates(k=3)

Job desciption: Physics Teacher
We are seeking a teacher with a strong theoretical background in Physics and experience managing educational laboratories. The ideal candidate will be able to design and conduct laboratory practices, including developing equipment and materials to enrich students' learning. Experience in organizing practical experiments to promote a deep understanding of theoretical concepts will be valued.
Requirements:
- Teaching degree in Physics or related fields.
- Previous experience managing and organizing school laboratories.
- Ability to develop laboratory practices and collaborate on creating new resources.

        Top 3 candidate summaries:

        **Revisión de RÉSUMÉ**

* Informática Educativa: se enfoca en la tecnología educativa y su implementación en el aula escolar
* Didáctica Específica y Prácticas de la Enseñanza II del Profesorado de Física de la FCEyN-UBA: se desarrollan unidades didácticas específicas para esta materia
* Unidad Didáctica diseñada 